In [4]:
import re
import pandas as pd
fileName=('BURecordBook.txt')
tourneys=[]
teamSet=set()
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    conf='Independent'
    for i in rows:
        row=i.split('\t')
        if(len(row[0])>7 and len(row)==1 and ('COACH' not in i and 'OVERALL' not in i and 'ECAC:' not in i and 'CAPTAIN' not in i and 'HOCKEY' not in i and 'NEIHL:' not in i and 'forfeit' not in i)):
            tourneys.append(row[0])
            
            
tourneyDict={}
for i in tourneys:
    if(i=='Key to Tournaments'):
        continue
    tourney=i.split(' ')
    tourneyDict[tourney[0]]=' '.join(tourney[1:])
tourneyDict['nc']='Non-Colleigate'
tourneyDict['Oly nc']='1932 NEAAU Olympic tryouts-Non-Colleigate'
tourneyDict['ex']='Exhibition'
tourneyDict['HF ex']='Hall of Fame Game-Exhibition'
tourneyDict['IB ex']='Ice Breaker-Exhibition'

In [5]:
import re
fileName=('BURecordBook.txt')
tourneys=[]
teamSet=set()
gameList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    conf='Independent'
    for i in rows:      
        if(len(i)==7):
            season=i
        coachSearch=re.search("COACH: (.*)",i)
        if coachSearch != None:
            coach=coachSearch.group(1)
        captSearch=re.search("CAPTAIN*: (.*)",i)
        if captSearch != None:
            capt=captSearch.group(1) 
            #print(capt)
        confSearch=re.search("(NEIHL|ECAC|HOCKEY EAST):",i)
        if confSearch != None:
            conf=confSearch.group(1)
            if(conf=='HOCKEY EAST'):
                conf='Hockey East'
        if(re.search('\*',i)!=None):
            gameType='Conference'
        else:
            gameType='Non-Conference'
        i=i.replace("* ",'')
        note=''
        if('†' in i):
            note='Loss by forfeit (ineligible player)'
            i=i.replace('†','')
        if('+' in i):
            note='Win by forfeit (opponent left ice – score was 5-1, BU)'
            i=i.replace('+','')
        if('‡' in i):
            note='Win by forfeit (ineligible player)'
            i=i.replace('‡','')
        game=re.search(r"(\d*\/\d*) (\w*) (?:\((.?ot)\))? ?(.*)\t(\S*|\S* \S*|\S* \S* \S*) ?(\(.*\))? (\d*-\d*)",i)
        if game==None:
            #print(i)
            continue
        #print(game.groups())
        gameDict={'date':game.group(1),
                 'result':game.group(2),
                 'ot':game.group(3),
                 'arena':game.group(4),
                 'opponent':game.group(5),
                 'gameType':gameType,
                 'tourney': game.group(6),
                 'scoreline':game.group(7),
                 'location':'',
                 'coach':coach,
                 'captain':capt,
                 'conference':conf,
                 'season':season,
                 'note':note}
        if(gameDict['gameType']==None):
            gameDict['gameType']='Non-Conference'
        if(gameDict['arena']=='Agganis Arena' or gameDict['arena']=='Brown Arena' or gameDict['arena']=='Boston Arena'):
            gameDict['location']='Home'
        elif(gameDict['tourney']==None or gameDict['tourney'] == '(B1G/HE)' or ((gameDict['tourney'] == '(HE)' or gameDict['tourney'] == '(ECAC)') and (gameDict['arena'] != 'TD Garden' and gameDict['arena'] != 'Boston Garden' and gameDict['arena'] != 'Providence CC'))):
            gameDict['location']='Away'
        if(gameDict['location']==''):
            gameDict['location']='Neutral'
        #if((gameDict['arena']=='Cross Arena' and gameDict['opponent']=='Maine') or (gameDict['arena']=='Gila River Arena' and gameDict['opponent']=='Arizona State') ):
        #    gameDict['location']='Neutral'
        if(gameDict['tourney']!=None):
            #print(gameDict)
            gameDict['tourney']=tourneyDict[gameDict['tourney'].replace('(','').replace(')','')]
        gameDict['month'],gameDict['day']=gameDict['date'].split('/')
        if(int(gameDict['month']) >=9):
            gameDict['date']+="/" + gameDict['season'][:4]
            gameDict['year']=gameDict['season'][:4]
        elif(int(gameDict['month']) <= 4):
            gameDict['date']+= "/" + str(int(gameDict['season'][:4])+1)
            gameDict['year']=int(gameDict['season'][:4])+1
        gameDict['BUScore'],gameDict['OppoScore']=int(gameDict['scoreline'].split('-')[0]),int(gameDict['scoreline'].split('-')[1])
        gameDict['GD']=abs(gameDict['BUScore']-gameDict['OppoScore'])
        gameList.append(gameDict)
        teamSet.add(gameDict['opponent'])
f.close()
df=pd.DataFrame(gameList)

In [65]:
teamList=teams.split('|')

In [64]:
teams="Air Force|Alaska Anchorage|American International|Arizona State|Army|Bates|Bentley|Boston College|Bowdoin|Bowling Green|Brown|Clarkson|Colby|Colgate|Colorado College|Connecticut|Cornell|Dalhousie|Dartmouth|Denver|Ferris State|Hamilton College|Harvard|Holy Cross|Lake Superior State|Loyola of Montreal|Maine|Massachusetts|Massachusetts Devens|UMass Lowell|Memorial University|Merrimack|Miami|Michigan|Michigan State|Michigan Tech|Middlebury|Minnesota|Minnesota Duluth|Minnesota State|MIT|Nebraska-Omaha|New Brunswick|New Hampshire|Niagara|North Dakota|Northeastern|Northern Arizona|Northern Michigan|Norwich|Notre Dame|Ohio State|Ohio University|Pennsylvania|Princeton|Providence|Queen’s|Quinnipiac|Rensselaer|Robert Morris|Royal Military College|Sacred Heart|St. Cloud State|St. Francis Xavier|St. Lawrence|St. Louis|Toronto|Tufts|Union|US International|Vermont|Western Michigan|Williams|Wisconsin|Yale|Intercollegiate Totals|Non-Collegiate|Boston Athletic Assoc.|Boston Hockey Club|Brae Burn|Briarcliff Lodge|Eastern Nationals|Jr. Olympics|L’Assoc. St. Dominique|Maple Athletic Assoc.|Needham Hockey Club|New Haven Club|Nichols Club|St. Nicholas|Tech Beavers|Union Boat Club|University Club|U.S. National Team"

In [7]:
#for s in ['2013-14','2014-15','2015-16','2016-17','2017-18']:
for s in ['2013-14']:

    print(s)
    for i in ['Home','Away','Neutral']:
        print(i,end=":")
        for d in ['W','L','T']:
            print(df.loc[(df['coach']=="David Quinn") & (df['location']==i) & (df['season']==s) & (df['result']==d) & (df['tourney']!='Exhibition')]['date'].count(),end='-')
        print("\b")

2013-14
Home:8-6-3-
Away:1-12-1-
Neutral:1-3-0-


21

In [6]:
df.loc[(df['coach']=="David Quinn")  & (df['season']=='2013-14')  & (df['result']=='W')  &  (df['tourney']!='Exhibition')].count()

date          10
result        10
ot             0
arena         10
opponent      10
gameType      10
tourney        2
scoreline     10
location      10
coach         10
captain       10
conference    10
season        10
note          10
month         10
day           10
year          10
BUScore       10
OppoScore     10
GD            10
dtype: int64

In [28]:
locs=set()
for i in gameList:
    locs.add(i['arena'])

In [46]:
locs

{'Agganis Arena',
 'Alfond (Colby)',
 'Alfond Arena',
 'Appleton',
 'BGSU Arena',
 'Baker Rink',
 'Bates Rink',
 'Bear Mountain',
 'Beaverbrook',
 'Beebe Lake',
 'Berry Events Center',
 'Boston Arena',
 'Boston Garden',
 'Bowdoin Rink',
 'Bradley Center',
 'Bright Center',
 'Broadmoor',
 'Brown Arena',
 'Cheel Arena',
 'Class of 1965 Arena',
 'Compton Arena',
 'Conte Forum',
 'Cross Arena',
 'Cumberland CC',
 'DCU Center',
 'DEC Center',
 'DU Arena',
 'Dane Cty. Col.',
 'Davis Rink',
 'Dayton Arena',
 'Engelstad Arena',
 'Fenway Park',
 'Four Seasons',
 'Freitas Ice Forum',
 'Gila River Arena',
 'Gutterson',
 'Hartford CC',
 'High Point Arena',
 'House of Ice',
 'Houston',
 'Ice Pavilion',
 'Ingalls Rink',
 'Joe Louis',
 'Kohl Center',
 'Lakeview Arena',
 'Lawler Arena',
 'Lawler Rink',
 'Lawlor Arena',
 'Lynah Rink',
 'MIT Rink',
 'MSG',
 'MSG III',
 'MSG IV',
 'Magness Arena',
 'Mariucci Arena',
 'Matthews Arena',
 'McHugh Forum',
 'Meehan',
 'Messa Rink',
 'Mullins Center',
 'Munn A